In [4]:
# Cell 1: Import all components
import sys
sys.path.append('..')

from src.document_processor import DocumentProcessor
from src.embeddings import EmbeddingsManager
from src.query_parser import Query_parser
import json

print("✅ All components imported")

✅ All components imported


In [5]:
print("Initializing components...")

doc_processor=DocumentProcessor()
em=EmbeddingsManager()
em.create_collection(collection_name="policy_documents")
query_parser=Query_parser()

print("✅ All components initialized")
print(f"   Vector store has {em.collection.count()} documents")

Initializing components...
✅ All components initialized
   Vector store has 0 documents


In [7]:
def test_end_to_end(query:str):
    print("="*80)
    print(f"Processing query:{query}")
    print("="*80+'\n')
    
    print("step 1: Parse the query..")
    parsed=query_parser.parse(query)
    print(f"  ✓ Age: {parsed.age}")
    print(f"  ✓ Gender: {parsed.gender}")
    print(f"  ✓ Procedure: {parsed.procedure}")
    print(f"  ✓ Location: {parsed.location}")
    print(f"  ✓ Duration: {parsed.policy_duration_months} months")
    print(f"  ✓ Emergency: {parsed.is_emergency}")
    
    validation = query_parser.validate_parsed_query(parsed)
    print(f"  ✓ Complete: {validation['is_complete']}\n")
    
    #search vector database
    print("Searching vector database for docs..")
    search_results=em.search(query,top_k=3)
    print(f"found {len(search_results['documents'])} relevent clauses")
    print("Step 3: Retrieved Policy Clauses:\n")
    for i, (doc, meta, dist) in enumerate(zip(
        search_results['documents'],
        search_results['metadatas'],
        search_results['distances']
    ), 1):
        print(f"  Clause {i}:")
        print(f"    Section: {meta['section']}")
        print(f"    Relevance: {1 - dist:.2%}")  # Convert distance to similarity %
        print(f"    Text: {doc[:150]}...")
        print()
    
    return {
        'parsed': parsed,
        'search_results': search_results,
        'validation': validation
    }
    

In [8]:
result1 = test_end_to_end("46 year old male, knee surgery in Pune, 3 month policy")

Processing query:46 year old male, knee surgery in Pune, 3 month policy

step 1: Parse the query..
  ✓ Age: 46
  ✓ Gender: male
  ✓ Procedure: knee surgery
  ✓ Location: Pune
  ✓ Duration: 3 months
  ✓ Emergency: False
  ✓ Complete: True

Searching vector database for docs..


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

found 0 relevent clauses
Step 3: Retrieved Policy Clauses:



In [9]:
result2 = test_end_to_end("35F hip surgery Mumbay 6 month policy")

Processing query:35F hip surgery Mumbay 6 month policy

step 1: Parse the query..
  ✓ Age: 35
  ✓ Gender: female
  ✓ Procedure: hip replacement
  ✓ Location: None
  ✓ Duration: 6 months
  ✓ Emergency: False
  ✓ Complete: False

Searching vector database for docs..


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

found 0 relevent clauses
Step 3: Retrieved Policy Clauses:



In [10]:
result3 = test_end_to_end("28M emergency cardiac procedure Bangalore 1 month")

Processing query:28M emergency cardiac procedure Bangalore 1 month

step 1: Parse the query..
  ✓ Age: 28
  ✓ Gender: male
  ✓ Procedure: cardiac
  ✓ Location: Bangalore
  ✓ Duration: 28 months
  ✓ Emergency: True
  ✓ Complete: True

Searching vector database for docs..


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

found 0 relevent clauses
Step 3: Retrieved Policy Clauses:



In [11]:
result4 = test_end_to_end("55M appendix surgery Delhi 5 month policy")

Processing query:55M appendix surgery Delhi 5 month policy

step 1: Parse the query..
  ✓ Age: 55
  ✓ Gender: male
  ✓ Procedure: appendix
  ✓ Location: Delhi
  ✓ Duration: 55 months
  ✓ Emergency: False
  ✓ Complete: True

Searching vector database for docs..


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

found 0 relevent clauses
Step 3: Retrieved Policy Clauses:



In [12]:
# Cell 8: Test Case 5 - Complex Query
result5 = test_end_to_end("Female patient aged 42 needs cataract surgery in Chennai, policy active for 12 months")


Processing query:Female patient aged 42 needs cataract surgery in Chennai, policy active for 12 months

step 1: Parse the query..
  ✓ Age: 12
  ✓ Gender: female
  ✓ Procedure: cataract
  ✓ Location: Chennai
  ✓ Duration: 12 months
  ✓ Emergency: False
  ✓ Complete: True

Searching vector database for docs..


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

found 0 relevent clauses
Step 3: Retrieved Policy Clauses:



In [19]:
# Cell 9: Analyze Search Quality
print("\n" + "="*80)
print("SEARCH QUALITY ANALYSIS")
print("="*80 + "\n")

test_cases = [
    ("knee surgery query", result1),
    ("hip surgery query", result2),
    ("cardiac query", result3),
    ("appendix query", result4),
    ("cataract query", result5),
]

for name, result in test_cases:
    top_section = result['search_results']['metadatas'][0]['section']
    top_distance = result['search_results']['distances'][0]
    
    print(f"{name:25} → Top match: {top_section[:30]:30} (dist: {top_distance:.3f})")



SEARCH QUALITY ANALYSIS



IndexError: list index out of range

In [18]:
print("\n" + "="*80)
print("DECISION PREVIEW (Manual Logic)")
print("="*80 + "\n")

def preview_decision(parsed, search_results):
    """
    Simple rule-based decision preview
    (Will be replaced with LLM on Day 6-7)
    """
    decision = {
        'approved': None,
        'reasoning': [],
        'checks': {}
    }
    
    # Check 1: Age eligibility (18-50)
    if parsed.age:
        if 18 <= parsed.age <= 50:
            decision['checks']['age'] = '✓ Pass'
            decision['reasoning'].append(f"Age {parsed.age} is within 18-50 range")
        else:
            decision['checks']['age'] = '✗ Fail'
            decision['reasoning'].append(f"Age {parsed.age} outside standard range")
    
    # Check 2: Policy duration (minimum 2 months for elective, immediate for emergency)
    if parsed.policy_duration_months:
        if parsed.is_emergency:
            decision['checks']['duration'] = '✓ Pass (Emergency)'
            decision['reasoning'].append("Emergency case - immediate coverage")
        elif parsed.policy_duration_months >= 2:
            decision['checks']['duration'] = '✓ Pass'
            decision['reasoning'].append(f"Policy duration {parsed.policy_duration_months} months meets minimum")
        else:
            decision['checks']['duration'] = '✗ Fail'
            decision['reasoning'].append(f"Policy duration {parsed.policy_duration_months} months below minimum")
    
    # Check 3: Location coverage
    tier1_cities = ['Mumbai', 'Delhi', 'Bangalore', 'Pune', 'Hyderabad', 'Chennai']
    if parsed.location:
        if parsed.location in tier1_cities:
            decision['checks']['location'] = '✓ Pass (100%)'
            decision['reasoning'].append(f"{parsed.location} is Tier 1 city - 100% coverage")
        else:
            decision['checks']['location'] = '✓ Pass (70-80%)'
            decision['reasoning'].append(f"{parsed.location} has 70-80% coverage")
    
    # Final decision
    all_pass = all(v.startswith('✓') for v in decision['checks'].values())
    decision['approved'] = all_pass
    
    return decision

# Test preview decisions
for name, result in test_cases[:3]:
    print(f"\n{name.upper()}:")
    decision = preview_decision(result['parsed'], result['search_results'])
    print(f"  Decision: {'APPROVED' if decision['approved'] else 'NEEDS REVIEW'}")
    print(f"  Checks:")
    for check, status in decision['checks'].items():
        print(f"    {check}: {status}")
    print(f"  Reasoning:")
    for reason in decision['reasoning']:
        print(f"    - {reason}")

print("\n✅ Integration test complete!")
print("📝 Ready for Day 6-7: Decision Engine with LLM")


DECISION PREVIEW (Manual Logic)


KNEE SURGERY QUERY:
  Decision: APPROVED
  Checks:
    age: ✓ Pass
    duration: ✓ Pass
    location: ✓ Pass (100%)
  Reasoning:
    - Age 46 is within 18-50 range
    - Policy duration 3 months meets minimum
    - Pune is Tier 1 city - 100% coverage

HIP SURGERY QUERY:
  Decision: APPROVED
  Checks:
    age: ✓ Pass
    duration: ✓ Pass
  Reasoning:
    - Age 35 is within 18-50 range
    - Policy duration 6 months meets minimum

CARDIAC QUERY:
  Decision: APPROVED
  Checks:
    age: ✓ Pass
    duration: ✓ Pass (Emergency)
    location: ✓ Pass (100%)
  Reasoning:
    - Age 28 is within 18-50 range
    - Emergency case - immediate coverage
    - Bangalore is Tier 1 city - 100% coverage

✅ Integration test complete!
📝 Ready for Day 6-7: Decision Engine with LLM
